In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:
df = pd.read_csv("data-dir/train_data.csv")
df = df.drop(df.loc[(df.duplicated(subset=["Question"], keep=False)) & (df["Answer_possible"] == False)].index, axis=0).reset_index(drop=True)
df

,Unnamed: 0,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start
0,2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],[526]
1,6,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],[505]
2,7,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360]
3,8,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],[276]
4,9,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290]
...,...,...,...,...,...,...,...
75030,130309,Matter,These quarks and leptons interact through four...,How many quarks and leptons are there?,False,[],[]
75031,130310,Matter,These quarks and leptons interact through four...,What model satisfactorily explains gravity?,False,[],[]
75032,130312,Matter,These quarks and leptons interact through four...,Mass and energy can always be compared to what?,False,[],[]
75033,130314,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,False,[],[]


In [5]:
ques2idx = {ques: str(idx) for idx, ques in enumerate(df["Question"].unique())}
idx2ques = {value: key for key, value in ques2idx.items()}

para2idx = {para: str(idx) for idx, para in enumerate(df["Paragraph"].unique())}
idx2para = {value: key for key, value in para2idx.items()}

theme2idx = {theme: str(idx) for idx, theme in enumerate(df["Theme"].unique())}
idx2theme = {value: key for key, value in theme2idx.items()}

paragraphs = df['Paragraph']
para_idx_2_theme_idx = {}
for idx, row in df.iterrows():
    para_idx_2_theme_idx[para2idx[row['Paragraph']]
                            ] = theme2idx[row['Theme']]

for i in tqdm(range(len(df))):
    df.loc[i, "question_id"] = ques2idx[df.iloc[i]["Question"]]
    df.loc[i, "paragraph_id"] = para2idx[df.iloc[i]["Paragraph"]]
    df.loc[i, "theme_id"] = theme2idx[df.iloc[i]["Theme"]]


100%|██████████| 75035/75035 [00:27<00:00, 2719.39it/s]


In [6]:
df

,Unnamed: 0,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start,question_id,paragraph_id,theme_id
0,2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],[526],0,0,0
1,6,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],[505],1,0,0
2,7,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360],2,0,0
3,8,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],[276],3,0,0
4,9,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290],4,0,0
...,...,...,...,...,...,...,...,...,...,...
75030,130309,Matter,These quarks and leptons interact through four...,How many quarks and leptons are there?,False,[],[],75014,15553,360
75031,130310,Matter,These quarks and leptons interact through four...,What model satisfactorily explains gravity?,False,[],[],75015,15553,360
75032,130312,Matter,These quarks and leptons interact through four...,Mass and energy can always be compared to what?,False,[],[],75016,15553,360
75033,130314,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,False,[],[],75017,15554,360
